In [ ]:
import numpy as np
from numpy import linspace, sqrt, sin, exp, convolve, abs, where

def rect(x,T):
    return where(abs(x)<=T/2, 1, 0)



In [ ]:
#Test rect
import plotly.graph_objects as go

fig = go.Figure()

y= np.ones(100)
x = np.arange(-50,50,1)
print(x.shape)
print(y.shape)
fig.add_trace(go.Scatter(x=x,y=y*rect(x-5,10)))

fig.show()

In [ ]:
#la fréquence evolue de omega_0-delta_omega/2 à omega_0+delta_omega/2
T = 0.2 #Le temps d'une chirp
delta_omega = 7000 #delta de frequence autour de omega_0
omega_0 = 3500 #fréquence de base
delta_f = 7000/(2*np.pi)
f_0 = 3500/(2*np.pi)
A=1 #amplitude du signal
P = 5 #nombre de chirp envoyé
Tr = T*1.2 #distance entre les chirp. Si Tr=T

t = np.linspace(0,1,10000)

e_c = lambda t : np.exp(1j*(-delta_omega/2 + delta_omega*t/(2*T))*t)

def e_s(t, P, Tr):
    out = 0
    for p in range(P):
        out += e_c(t-p*Tr)*np.exp(-1j*omega_0*p*Tr)*rect(t-p*Tr,T)
    return out

def window(t,p,Tr):
    out = 0
    for p in range(P):
        out += rect(t-p*Tr,T)
    return out

def e_s_without_win(t,P,Tr):
    out = 0
    out = e_c(t)*rect(t,P*T)
    return out

s_e = e_s(t,P,Tr)*np.exp(1j*omega_0*t)
win = window(t,P,Tr)
s_e_without_win = e_s_without_win(t,P,Tr)*np.exp(1j*omega_0*t)
#print(np.real(s_e))

fig = go.Figure()
fig.add_trace(go.Scatter(x=t,y=np.real(s_e)))
fig.add_trace(go.Scatter(x=t,y=win))
fig.show()



In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t,y=np.real(s_e)))
fig.add_trace(go.Scatter(x=t,y=win))
fig.show()

from scipy import signal
se_fft = np.fft.fft(s_e)
f, t, Zxx =signal.stft(s_e,10000,nperseg=200,return_onesided=False)

print(f.shape)
print(t.shape)

fig = go.Figure()


#fig.add_trace(go.Heatmap(x0=0,dx=0.0482,y0=-40000,dy=8,z=np.abs(Zxx)[np.argsort(f)],zmin=0,zmax=0.25,zsmooth = 'best'))
fig.add_trace(go.Heatmap(x=t,y=f[np.argsort(f)],z=np.abs(Zxx)[np.argsort(f)],zmin=0,zmax=0.25,zsmooth = 'best',colorbar=dict(title="Amplitude")))

#print(f[index])
fig.update_layout(yaxis_range=[-1000,1000], xaxis_title="Time [s]", yaxis_title="Frequency [Hz]",title_text="STFT of a moving chair @2.4GHz")
fig.show()

# Signal reçu

In [ ]:
v = 1e4 #[m/s]
d = 1.5*3e6

beta = 2*v/3e8
alpha = 2*d/3e8

r = A*np.exp(1j*omega_0*t*(1-beta))*e_s(t-alpha,P,Tr)*np.exp(-1j*omega_0*alpha)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t,y=np.real(r)))
fig.add_trace(go.Scatter(x=t,y=win))
fig.show()

fig = go.Figure()

#fig.add_trace(go.Heatmap(x0=0,dx=0.0482,y0=-40000,dy=8,z=np.abs(Zxx)[np.argsort(f)],zmin=0,zmax=0.25,zsmooth = 'best'))
fig.add_trace(go.Heatmap(x=t,y=f[np.argsort(f)],z=np.abs(Zxx)[np.argsort(f)],zmin=0,zmax=0.25,zsmooth = 'best',colorbar=dict(title="Amplitude")))

#print(f[index])
fig.update_layout(yaxis_range=[-1000,1000], xaxis_title="Time [s]", yaxis_title="Frequency [Hz]",title_text="STFT of a moving chair @2.4GHz")
fig.show()

In [ ]:
from scipy import signal
import plotly.graph_objects as go
from plotly.offline import plot

import numpy as np

t = np.linspace(0,1,500)

f = 10 #Hz
phi = -90 #degrees

squre_1 = 7.5 * signal.square(2*np.pi*f*t) + 7.5
squre_2 = 7.5 * signal.square(2*np.pi*f*t + np.pi/2 + np.pi/180 * phi) + 7.5

phase_diff = np.abs(squre_1-squre_2)

fig = go.Figure()
for d_phi in np.arange(-90,90,1):
    squre_2 = 7.5 * signal.square(2*np.pi*f*t + np.pi/2 + np.pi/180 * d_phi) + 7.5
    phase_diff = np.abs(squre_1-squre_2)
    phase_diff_avg = np.average(phase_diff)
    side = 'right' if phase_diff_avg<7.5 else 'left'
    fig.add_trace(
        go.Scatter(visible=False, 
        name= "Angle: {}, Avg:{} signal comes from {} side".format(d_phi,phase_diff_avg,side),
        x=t, 
        y=phase_diff))


fig.data[45].visible = True

steps = []

for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        label=str(i*180/len(fig.data)-90),
        args=["visible", [False] * len(fig.data)],
    )
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=45,
    currentvalue={"prefix": "angle: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    title="Side of incoming sound + Angle",
    sliders=sliders,
    showlegend=True)
#plot(fig, filename='fig.html')
fig['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]
fig.show()

#plt.plot(t,squre_1)
#plt.figure()
#plt.subplot(2,1,1)
#plt.plot(t,squre_2 - squre_1)
#plt.subplot(2,1,2)
#plt.plot(t, phase_diff)

#print(np.average(np.abs(squre_1-squre_2)))
#plt.show()




In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import scipy.signal as scsa

data = np.load('group1_bras_3.npz')
f0 = data[data.files[2]][0]
B = data[data.files[2]][1]
Ns = int(data[data.files[2]][2])
Nr = int(data[data.files[2]][3])
Ts = data[data.files[2]][4]
Tr = data[data.files[2]][5]
 
c = 3*10**8
 
x = np.linspace(0, Ns*Nr*Ts , int(Ns*Nr)) 
d = np.linspace(0,50,50).reshape((50,1))
dx = np.linspace(0,50,50)
vx = np.linspace(-10,10,50)
v = np.linspace(-10,10,50).reshape((1,50))
 
bg = data[data.files[1]].reshape((Nr*Ns,1))
l = []

X, Y = np.meshgrid(d, v,)

fig = go.Figure()


vf = np.vectorize(f,excluded=[2])
for i in range(data[data.files[0]].shape[0]):
    frame = (data[data.files[0]][i].reshape((Nr*Ns,1)) - bg).reshape(Ns,Nr)
#     envelope = np.abs(scsa.hilbert(np.abs(frame)))
#     C = np.zeros((d.shape[0],v.shape[1]),dtype='complex128')
#     print(C.shape)
#     for n2 in range(0,int(Ns)):
#         for n1 in range(0,int(Nr)):
#             C += np.dot(np.e**(1j * d*2/c * B * (n2*Ts/Tr)),np.e**(1j * n1 * Nr*Ts*2*np.pi*f0*2*v/c))*frame[n1*int(Ns) + n2]
#     C = np.power(abs(C),2)
    C = np.fft.fft2(frame)
    print(C)
    #print(frame.shape)
    # l.append(C.T)
    # fig.add_trace(go.Heatmap(visible=False, x=dx,y=vx[np.argsort(vx)],z=np.abs(l[i])[np.argsort(vx)],zsmooth = 'best',colorbar=dict(title="Amplitude")))


fig.data[45].visible = True

steps = []

for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        label=str(i/len(fig.data)),
        args=["visible", [False] * len(fig.data)], 
    )
    step["args"][1][i] = True
    steps.append(step)

sliders = [dict(
    active=45,
    currentvalue={"prefix": "angle: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    title="Side of incoming sound + Angle",
    sliders=sliders,
    showlegend=True)

fig.update_layout(yaxis_range=[-10,10], xaxis_title="Distance [m]", yaxis_title="Speed [m/s]",title_text="STFT of a moving chair @2.4GHz")
fig.show()


In [ ]:
d = 1
v = 1

simil_frame = np.zeros(Ns*Nr,dtype ='complex128' )
for n2 in range(0,int(Ns)):
    for n1 in range(0,int(Nr)):
        simil_frame[n1*Ns + n2] = np.dot(np.e**(1j * d*2/c * B * (n2*Ts/Tr)),np.e**(1j * n2 * Ts *2*np.pi*f0*2*v/c + 1j * n1 * Nr*Ts*2*np.pi*f0*2*v/c))


plt.plot(np.linspace(0, Ns*Nr*Ts , int(Ns*Nr)),simil_frame)
plt.plot(np.linspace(0, Ns*Nr*Ts , int(Ns*Nr)),scsa.hilbert(np.abs(data[data.files[0]][0].reshape((Nr*Ns,1)) - bg)))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.animation import FuncAnimation
import scipy.signal as scsa
import numpy as np
data = np.load('group1_clean_1.npz')
f0 = data[data.files[2]][0]
B = data[data.files[2]][1]
Ns = int(data[data.files[2]][2])
Nr = int(data[data.files[2]][3])
Ts = data[data.files[2]][4]
Tr = data[data.files[2]][5]
 
c = 3*10**8

bg = data[data.files[1]].reshape((Nr*Ns,1))
l = []

for i in range(0,data[data.files[0]].shape[0]):
    frame = (data[data.files[0]][i].reshape((Nr*Ns,1)) - bg).reshape(Nr,Ns)
    frame = frame - np.mean(frame)
    C = np.fft.fft2(frame,s = (200,200))
    C = np.fft.fftshift(C)
    print(C.shape)
    v = np.linspace(-1,1,C.shape[0])*c/(2*B)
    d = np.linspace(0,1,C.shape[1])*( c*  np.pi* 1/Ts * 2 / ( 2 * 2*np.pi * f0 * Ns ) )
    X, Y = np.meshgrid(d, v,)
    print(X.shape,Y.shape)
    fig, ax = plt.subplots()
    ax.pcolormesh(X, Y, abs(C))
    plt.savefig("abc" + str(i) + ".png")
    plt.close()